In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark= SparkSession \
       .builder \
       .appName("daily_practice") \
       .config('spark.ui.port', '4050') \
       .getOrCreate()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,420 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [55.7 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updat

In [2]:
from google.colab import output
output.serve_kernel_port_as_window(4050, path ='/jobs/index.html')

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# 🚩 **Day 7 - 2025/05/02**

### Q1: Window Function for Moving Calculations
Calculate a moving average of stock prices for multiple companies. For each stock, show the current day's closing price along with the 3-day and 7-day moving averages.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

# Sample stock price data
stock_data = spark.createDataFrame([
    ("AAPL", "2024-04-01", 175.20),
    ("AAPL", "2024-04-02", 177.50),
    ("AAPL", "2024-04-03", 176.80),
    ("AAPL", "2024-04-04", 179.30),
    ("AAPL", "2024-04-05", 180.25),
    ("AAPL", "2024-04-08", 182.15),
    ("AAPL", "2024-04-09", 181.45),
    ("AAPL", "2024-04-10", 183.75),
    ("MSFT", "2024-04-01", 410.50),
    ("MSFT", "2024-04-02", 412.80),
    ("MSFT", "2024-04-03", 408.25),
    ("MSFT", "2024-04-04", 415.75),
    ("MSFT", "2024-04-05", 417.90),
    ("MSFT", "2024-04-08", 420.10),
    ("MSFT", "2024-04-09", 419.25),
    ("MSFT", "2024-04-10", 422.50)
], ["ticker", "date", "close_price"])
```

Expected output:
```
+-------+----------+-----------+------------------+------------------+
|ticker |date      |close_price|moving_avg_3day   |moving_avg_7day   |
+-------+----------+-----------+------------------+------------------+
|AAPL   |2024-04-10|183.75     |182.45            |181.02            |
|AAPL   |2024-04-09|181.45     |181.28            |179.61            |
|AAPL   |2024-04-08|182.15     |180.88            |178.74            |
|...    |...       |...        |...               |...               |
|MSFT   |2024-04-10|422.50     |420.62            |416.68            |
|...    |...       |...        |...               |...               |
+-------+----------+-----------+------------------+------------------+
```


In [5]:
# current day's closing price along with the 3-day and 7-day moving averages.
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

# Sample stock price data
stock_data = spark.createDataFrame([
    ("AAPL", "2024-04-01", 175.20),
    ("AAPL", "2024-04-02", 177.50),
    ("AAPL", "2024-04-03", 176.80),
    ("AAPL", "2024-04-04", 179.30),
    ("AAPL", "2024-04-05", 180.25),
    ("AAPL", "2024-04-08", 182.15),
    ("AAPL", "2024-04-09", 181.45),
    ("AAPL", "2024-04-10", 183.75),
    ("MSFT", "2024-04-01", 410.50),
    ("MSFT", "2024-04-02", 412.80),
    ("MSFT", "2024-04-03", 408.25),
    ("MSFT", "2024-04-04", 415.75),
    ("MSFT", "2024-04-05", 417.90),
    ("MSFT", "2024-04-08", 420.10),
    ("MSFT", "2024-04-09", 419.25),
    ("MSFT", "2024-04-10", 422.50)
], ["ticker", "date", "close_price"])

# last two prices and current price
window_3_day = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.currentRow - 2, Window.currentRow)

# last six prices and current price
window_7_day = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.currentRow - 6, Window.currentRow)


stock_data = stock_data.withColumn("date", F.to_date("date"))

moving_averages = stock_data.withColumn("moving_avg_3day", F.round(F.avg("close_price").over(window_3_day), 2))\
                            .withColumn("moving_avg_7day", F.round(F.avg("close_price").over(window_7_day), 2))\
                            .select("ticker", "date", "close_price", "moving_avg_3day", "moving_avg_7day").orderBy(F.col("ticker").asc(), F.col("date").desc())

moving_averages.show()


+------+----------+-----------+---------------+---------------+
|ticker|      date|close_price|moving_avg_3day|moving_avg_7day|
+------+----------+-----------+---------------+---------------+
|  AAPL|2024-04-10|     183.75|         182.45|         180.17|
|  AAPL|2024-04-09|     181.45|         181.28|         178.95|
|  AAPL|2024-04-08|     182.15|         180.57|         178.53|
|  AAPL|2024-04-05|     180.25|         178.78|         177.81|
|  AAPL|2024-04-04|      179.3|         177.87|          177.2|
|  AAPL|2024-04-03|      176.8|          176.5|          176.5|
|  AAPL|2024-04-02|      177.5|         176.35|         176.35|
|  AAPL|2024-04-01|      175.2|          175.2|          175.2|
|  MSFT|2024-04-10|      422.5|         420.62|         416.65|
|  MSFT|2024-04-09|     419.25|         419.08|         414.94|
|  MSFT|2024-04-08|      420.1|         417.92|         414.22|
|  MSFT|2024-04-05|      417.9|         413.97|         413.04|
|  MSFT|2024-04-04|     415.75|         

### Q2: Data Deduplication with Business Rules
Remove duplicate customer records based on complex business rules: if multiple records exist for the same customer, keep the most complete record with preference to more recent updates.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, coalesce, sum, max, row_number
from pyspark.sql.window import Window

# Sample customer data with duplicates and varying completeness
customer_data = spark.createDataFrame([
    (1001, "John", "Smith", "john.smith@email.com", "123-456-7890", "123 Main St", "New York", "NY", "2024-01-15"),
    (1001, "John", "Smith", "john.smith@email.com", None, "123 Main St", "New York", "NY", "2024-02-20"),
    (1001, "John", "Smith", "john.s@email.com", "123-456-7890", "123 Main St", None, "NY", "2024-03-05"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", None, "Los Angeles", "CA", "2024-01-10"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", "456 Oak Ave", "Los Angeles", "CA", "2024-02-25"),
    (1003, "Michael", "Johnson", "michael.j@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-01-20"),
    (1003, "Michael", "Johnson", "michael.johnson@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-02-10")
], ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "last_updated"])
```

Expected output:
```
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
|customer_id |first_name|last_name|email                 |phone        |address      |city         |state|last_updated|
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
|1001        |John      |Smith    |john.smith@email.com  |123-456-7890 |123 Main St  |New York     |NY   |2024-03-05  |
|1002        |Jane      |Doe      |jane.doe@email.com    |987-654-3210 |456 Oak Ave  |Los Angeles  |CA   |2024-02-25  |
|1003        |Michael   |Johnson   |michael.johnson@email.com|555-123-4567|789 Pine St|Chicago      |IL   |2024-02-10  |
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
```

In [14]:
from pyspark.sql.functions import col, count, when, coalesce, sum, max, row_number
from pyspark.sql.window import Window

# Sample customer data with duplicates and varying completeness
customer_data = spark.createDataFrame([
    (1001, "John", "Smith", "john.smith@email.com", "123-456-7890", "123 Main St", "New York", "NY", "2024-01-15"),
    (1001, "John", "Smith", "john.smith@email.com", None, "123 Main St", "New York", "NY", "2024-02-20"),
    (1001, "John", "Smith", "john.s@email.com", "123-456-7890", "123 Main St", None, "NY", "2024-03-05"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", None, "Los Angeles", "CA", "2024-01-10"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", "456 Oak Ave", "Los Angeles", "CA", "2024-02-25"),
    (1003, "Michael", "Johnson", "michael.j@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-01-20"),
    (1003, "Michael", "Johnson", "michael.johnson@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-02-10")
], ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "last_updated"])

customer_data = customer_data.withColumn("last_updated", F.to_date("last_updated"))

customer_data.select("*").orderBy(F.col("customer_id").asc(), F.col("last_updated").desc()).show()


+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|customer_id|first_name|last_name|               email|       phone|    address|       city|state|last_updated|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|       1001|      John|    Smith|    john.s@email.com|123-456-7890|123 Main St|       NULL|   NY|  2024-03-05|
|       1001|      John|    Smith|john.smith@email.com|        NULL|123 Main St|   New York|   NY|  2024-02-20|
|       1001|      John|    Smith|john.smith@email.com|123-456-7890|123 Main St|   New York|   NY|  2024-01-15|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|456 Oak Ave|Los Angeles|   CA|  2024-02-25|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|       NULL|Los Angeles|   CA|  2024-01-10|
|       1003|   Michael|  Johnson|michael.johnson@e...|555-123-4567|789 Pine St|    Chicago|   IL|  2024

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Step 1: Calculate completeness score
df_with_score = customer_data.withColumn("completeness_score",
    (F.col("customer_id").isNotNull().cast("int") +
     F.col("first_name").isNotNull().cast("int") +
     F.col("last_name").isNotNull().cast("int") +
     F.col("email").isNotNull().cast("int") +
     F.col("phone").isNotNull().cast("int") +
     F.col("address").isNotNull().cast("int") +
     F.col("city").isNotNull().cast("int") +
     F.col("state").isNotNull().cast("int") +
     F.col("last_updated").isNotNull().cast("int"))
)

# Step 2: Define window spec - ensure last_updated is correctly ordered
window_spec = Window.partitionBy("customer_id")\
                   .orderBy(F.col("completeness_score").desc(),
                            F.to_date(F.col("last_updated"), "yyyy-MM-dd").desc())

# Step 3: Rank the records
df_ranked = df_with_score.withColumn("rank", F.row_number().over(window_spec))

# Step 4: Filter to keep only the top-ranked record
df_deduplicated = df_ranked.filter(F.col("rank") == 1).drop("completeness_score", "rank")

In [16]:
df_deduplicated.show()

+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|customer_id|first_name|last_name|               email|       phone|    address|       city|state|last_updated|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|       1001|      John|    Smith|john.smith@email.com|123-456-7890|123 Main St|   New York|   NY|  2024-01-15|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|456 Oak Ave|Los Angeles|   CA|  2024-02-25|
|       1003|   Michael|  Johnson|michael.johnson@e...|555-123-4567|789 Pine St|    Chicago|   IL|  2024-02-10|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+



### Q3: Dynamic Pivot and Unpivot
Create a solution that can dynamically pivot and unpivot data without hardcoding column names. Transform sales data from a normalized format to a crosstab view by product category and quarter, then transform it back.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, lit, concat, collect_list, struct, first, avg
import pyspark.sql.functions as F

# Sample sales data in normalized format
sales_data = spark.createDataFrame([
    ("Electronics", "Q1", 1250000),
    ("Electronics", "Q2", 1380000),
    ("Electronics", "Q3", 1120000),
    ("Electronics", "Q4", 1680000),
    ("Clothing", "Q1", 850000),
    ("Clothing", "Q2", 940000),
    ("Clothing", "Q3", 1020000),
    ("Clothing", "Q4", 1350000),
    ("Home Goods", "Q1", 720000),
    ("Home Goods", "Q2", 680000),
    ("Home Goods", "Q3", 790000),
    ("Home Goods", "Q4", 950000),
    ("Sports", "Q1", 450000),
    ("Sports", "Q2", 520000),
    ("Sports", "Q3", 670000),
    ("Sports", "Q4", 540000)
], ["category", "quarter", "sales"])

# You need to create two transformations:
# 1. Pivoted data - categories as rows, quarters as columns
# 2. Unpivoted data - return to original format from the pivoted data
```

Expected output for pivoted data:
```
+-------------+--------+--------+--------+--------+
|category     |Q1_sales|Q2_sales|Q3_sales|Q4_sales|
+-------------+--------+--------+--------+--------+
|Electronics  |1250000 |1380000 |1120000 |1680000 |
|Clothing     |850000  |940000  |1020000 |1350000 |
|Home Goods   |720000  |680000  |790000  |950000  |
|Sports       |450000  |520000  |670000  |540000  |
+-------------+--------+--------+--------+--------+
```

In [23]:
from pyspark.sql.functions import col, expr, lit, concat, collect_list, struct, first, avg
import pyspark.sql.functions as F

# Sample sales data in normalized format
sales_data = spark.createDataFrame([
    ("Electronics", "Q1", 1250000),
    ("Electronics", "Q2", 1380000),
    ("Electronics", "Q3", 1120000),
    ("Electronics", "Q4", 1680000),
    ("Clothing", "Q1", 850000),
    ("Clothing", "Q2", 940000),
    ("Clothing", "Q3", 1020000),
    ("Clothing", "Q4", 1350000),
    ("Home Goods", "Q1", 720000),
    ("Home Goods", "Q2", 680000),
    ("Home Goods", "Q3", 790000),
    ("Home Goods", "Q4", 950000),
    ("Sports", "Q1", 450000),
    ("Sports", "Q2", 520000),
    ("Sports", "Q3", 670000),
    ("Sports", "Q4", 540000)
], ["category", "quarter", "sales"])

sales_data.show()

# 1. Pivoted data - categories as rows, quarters as columns
sales_data_pivoted = sales_data.groupBy("category").pivot("quarter").agg(F.sum("sales"))
sales_data_pivoted.show()

+-----------+-------+-------+
|   category|quarter|  sales|
+-----------+-------+-------+
|Electronics|     Q1|1250000|
|Electronics|     Q2|1380000|
|Electronics|     Q3|1120000|
|Electronics|     Q4|1680000|
|   Clothing|     Q1| 850000|
|   Clothing|     Q2| 940000|
|   Clothing|     Q3|1020000|
|   Clothing|     Q4|1350000|
| Home Goods|     Q1| 720000|
| Home Goods|     Q2| 680000|
| Home Goods|     Q3| 790000|
| Home Goods|     Q4| 950000|
|     Sports|     Q1| 450000|
|     Sports|     Q2| 520000|
|     Sports|     Q3| 670000|
|     Sports|     Q4| 540000|
+-----------+-------+-------+

+-----------+-------+-------+-------+-------+
|   category|     Q1|     Q2|     Q3|     Q4|
+-----------+-------+-------+-------+-------+
|     Sports| 450000| 520000| 670000| 540000|
|Electronics|1250000|1380000|1120000|1680000|
|   Clothing| 850000| 940000|1020000|1350000|
| Home Goods| 720000| 680000| 790000| 950000|
+-----------+-------+-------+-------+-------+



In [27]:
# 2. Unpivoted data - return to original format from the pivoted data
sales_data_pivoted.unpivot("category", ["Q1", "Q2", "Q3", "Q4"],"quarter", "sales").show()

+-----------+-------+-------+
|   category|quarter|  sales|
+-----------+-------+-------+
|     Sports|     Q1| 450000|
|     Sports|     Q2| 520000|
|     Sports|     Q3| 670000|
|     Sports|     Q4| 540000|
|Electronics|     Q1|1250000|
|Electronics|     Q2|1380000|
|Electronics|     Q3|1120000|
|Electronics|     Q4|1680000|
|   Clothing|     Q1| 850000|
|   Clothing|     Q2| 940000|
|   Clothing|     Q3|1020000|
|   Clothing|     Q4|1350000|
| Home Goods|     Q1| 720000|
| Home Goods|     Q2| 680000|
| Home Goods|     Q3| 790000|
| Home Goods|     Q4| 950000|
+-----------+-------+-------+



### Q4: Sessionization with Complex Rules
Implement a sessionization algorithm that groups user activity into sessions. A new session starts after 30 minutes of inactivity OR when a user switches device types, regardless of time.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, when, sum, lit, coalesce, unix_timestamp, row_number
from pyspark.sql.window import Window

# Sample user activity data
user_logs = spark.createDataFrame([
    ("user1", "2024-04-01 10:15:20", "mobile", "/home"),
    ("user1", "2024-04-01 10:18:35", "mobile", "/products"),
    ("user1", "2024-04-01 10:25:42", "mobile", "/product/123"),
    ("user1", "2024-04-01 10:30:21", "desktop", "/product/123"),  # Device change
    ("user1", "2024-04-01 10:32:15", "desktop", "/cart"),
    ("user1", "2024-04-01 10:35:10", "desktop", "/checkout"),
    ("user1", "2024-04-01 11:20:30", "desktop", "/home"),  # Time gap
    ("user2", "2024-04-01 14:05:12", "tablet", "/home"),
    ("user2", "2024-04-01 14:08:25", "tablet", "/search"),
    ("user2", "2024-04-01 14:12:30", "tablet", "/product/456"),
    ("user2", "2024-04-01 14:45:12", "tablet", "/product/789"),  # Time gap
    ("user2", "2024-04-01 14:50:30", "mobile", "/product/789"),  # Device change
    ("user2", "2024-04-01 14:52:45", "mobile", "/cart")
], ["user_id", "timestamp", "device", "page"])
```

Expected output:
```
+-------+-------------------+-------+-------------+----------+--------------------+
|user_id|timestamp          |device |page         |session_id|session_duration_min|
+-------+-------------------+-------+-------------+----------+--------------------+
|user1  |2024-04-01 10:15:20|mobile |/home        |1         |10.37               |
|user1  |2024-04-01 10:18:35|mobile |/products    |1         |10.37               |
|user1  |2024-04-01 10:25:42|mobile |/product/123 |1         |10.37               |
|user1  |2024-04-01 10:30:21|desktop|/product/123 |2         |4.82                |
|user1  |2024-04-01 10:32:15|desktop|/cart        |2         |4.82                |
|user1  |2024-04-01 10:35:10|desktop|/checkout    |2         |4.82                |
|user1  |2024-04-01 11:20:30|desktop|/home        |3         |0.00                |
|user2  |2024-04-01 14:05:12|tablet |/home        |1         |7.30                |
|user2  |2024-04-01 14:08:25|tablet |/search      |1         |7.30                |
|user2  |2024-04-01 14:12:30|tablet |/product/456 |1         |7.30                |
|user2  |2024-04-01 14:45:12|tablet |/product/789 |2         |0.00                |
|user2  |2024-04-01 14:50:30|mobile |/product/789 |3         |2.25                |
|user2  |2024-04-01 14:52:45|mobile |/cart        |3         |2.25                |
+-------+-------------------+-------+-------------+----------+--------------------+
```